# 02. Representation of Data (for Machine Learning)

Machine learning is about creating models from data: for that reason, we'll start by
discussing how data can be represented in order to be understood by the computer.  Along
with this, we'll build on our matplotlib examples from the previous section and show some
examples of how to visualize data.

## Data in scikit-learn

Data in scikit-learn, with very few exceptions, is assumed to be stored as a
**two-dimensional array**, of size `[n_samples, n_features]`. 
This array is usually referrred as the **feature matrix**.

There is also the **label vector**, of size `n_samples`, containing the list of labels
for each samples (_Note_: **ONLY** in the Supervised Learning settings)

$$
{\rm feature~matrix:~~~} {\bf X}~=~\left[
\begin{matrix}
x_{11} & x_{12} & \cdots & x_{1D}\\
x_{21} & x_{22} & \cdots & x_{2D}\\
x_{31} & x_{32} & \cdots & x_{3D}\\
\vdots & \vdots & \ddots & \vdots\\
\vdots & \vdots & \ddots & \vdots\\
x_{N1} & x_{N2} & \cdots & x_{ND}\\
\end{matrix}
\right]
$$

$$
{\rm label~vector:~~~} {\bf y}~=~ [y_1, y_2, y_3, \cdots y_N]
$$

Here there are $N$ samples and $D$ features.

- $N$ (`n_samples`):   The number of samples: each sample is an item to process (e.g. classify).
  A sample can be a document, a picture, a sound, a video, an astronomical object,
  a row in database or CSV file,
  or whatever you can describe with a fixed set of quantitative traits.
- $D$ (`n_features`):  The number of features or distinct traits that can be used to describe each
  item in a quantitative manner.  Features are generally real-valued, but may be boolean or
  discrete-valued in some cases.

The number of features must be fixed in advance. 

However it can be very high dimensional
(e.g. millions of features) with most of them being zeros for a given sample. This is a case
where `scipy.sparse` matrices can be useful, in that they are
much more memory-efficient than numpy arrays.

Each sample (data point) is a row in the data array, and each feature is a column.

---

# Scipy Sparse Matrices

**Sparse Matrices** are very nice in some situations.  

For example, in some machine learning tasks, especially those associated
with textual analysis, the data may be mostly zeros.  

Storing all these zeros is very inefficient.  

We can create and manipulate sparse matrices as follows:

In [1]:
import numpy as np
np.random.seed(42)

In [2]:
# Create a random array with a lot of zeros
X = np.random.random((10, 5))
print(X)

[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184 0.45606998]
 [0.78517596 0.19967378 0.51423444 0.59241457 0.04645041]
 [0.60754485 0.17052412 0.06505159 0.94888554 0.96563203]
 [0.80839735 0.30461377 0.09767211 0.68423303 0.44015249]
 [0.12203823 0.49517691 0.03438852 0.9093204  0.25877998]
 [0.66252228 0.31171108 0.52006802 0.54671028 0.18485446]]


In [3]:
X[X < 0.7] = 0
print(X)

[[0.         0.95071431 0.73199394 0.         0.        ]
 [0.         0.         0.86617615 0.         0.70807258]
 [0.         0.96990985 0.83244264 0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.78517596 0.         0.         0.         0.        ]
 [0.         0.         0.         0.94888554 0.96563203]
 [0.80839735 0.         0.         0.         0.        ]
 [0.         0.         0.         0.9093204  0.        ]
 [0.         0.         0.         0.         0.        ]]


In [4]:
from scipy import sparse

# turn X into a csr (Compressed-Sparse-Row) matrix
X_csr = sparse.csr_matrix(X)
print(X_csr)

  (0, 1)	0.9507143064099162
  (0, 2)	0.7319939418114051
  (1, 2)	0.8661761457749352
  (1, 4)	0.7080725777960455
  (2, 1)	0.9699098521619943
  (2, 2)	0.8324426408004217
  (5, 0)	0.7851759613930136
  (6, 3)	0.9488855372533332
  (6, 4)	0.9656320330745594
  (7, 0)	0.8083973481164611
  (8, 3)	0.9093204020787821


In [5]:
# convert the sparse matrix to a dense array
print(X_csr.toarray())

[[0.         0.95071431 0.73199394 0.         0.        ]
 [0.         0.         0.86617615 0.         0.70807258]
 [0.         0.96990985 0.83244264 0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.78517596 0.         0.         0.         0.        ]
 [0.         0.         0.         0.94888554 0.96563203]
 [0.80839735 0.         0.         0.         0.        ]
 [0.         0.         0.         0.9093204  0.        ]
 [0.         0.         0.         0.         0.        ]]


In [6]:
# Sparse matrices support linear algebra:
y = np.random.random(X_csr.shape[1])
z1 = X_csr.dot(y)
z2 = X.dot(y)
np.allclose(z1, z2)

True

* The CSR representation can be very efficient for computations, but it is not as good for adding elements.  

* For that, the **LIL** (List-In-List) representation is better:

In [7]:
# Create an empty LIL matrix and add some items
X_lil = sparse.lil_matrix((5, 5))

for i, j in np.random.randint(0, 5, (15, 2)):
    X_lil[i, j] = i + j

print(X_lil)
print('\n')
print(X_lil.toarray())

  (0, 2)	2.0
  (0, 3)	3.0
  (0, 4)	4.0
  (1, 0)	1.0
  (1, 1)	2.0
  (1, 4)	5.0
  (2, 0)	2.0
  (2, 2)	4.0
  (2, 4)	6.0
  (3, 0)	3.0
  (3, 3)	6.0
  (3, 4)	7.0
  (4, 0)	4.0
  (4, 4)	8.0


[[0. 0. 2. 3. 4.]
 [1. 2. 0. 0. 5.]
 [2. 0. 4. 0. 6.]
 [3. 0. 0. 6. 7.]
 [4. 0. 0. 0. 8.]]


* Often, once an LIL matrix is created, it is useful to convert it to a CSR format 
    * **Note**: many scikit-learn algorithms require CSR or CSC format

In [8]:
X_csr = X_lil.tocsr()
print(X_csr)

  (0, 2)	2.0
  (0, 3)	3.0
  (0, 4)	4.0
  (1, 0)	1.0
  (1, 1)	2.0
  (1, 4)	5.0
  (2, 0)	2.0
  (2, 2)	4.0
  (2, 4)	6.0
  (3, 0)	3.0
  (3, 3)	6.0
  (3, 4)	7.0
  (4, 0)	4.0
  (4, 4)	8.0


There are several other sparse formats that can be useful for various problems:

- `CSC` (compressed sparse column)
- `BSR` (block sparse row)
- `COO` (coordinate)
- `DIA` (diagonal)
- `DOK` (dictionary of keys)

## CSC - Compressed Sparse Column

**Advantages of the CSC format**

    * efficient arithmetic operations CSC + CSC, CSC * CSC, etc.
    * efficient column slicing
    * fast matrix vector products (CSR, BSR may be faster)

**Disadvantages of the CSC format**

    * slow row slicing operations (consider CSR)
    * changes to the sparsity structure are expensive (consider LIL or DOK)

### BSR - Block Sparse Row

The Block Compressed Row (`BSR`) format is very similar to the Compressed Sparse Row (`CSR`) format. 

BSR is appropriate for sparse matrices with *dense sub matrices* like the example below. 

Block matrices often arise in *vector-valued* finite element discretizations. 

In such cases, BSR is **considerably more efficient** than CSR and CSC for many sparse arithmetic operations.

In [13]:
from scipy.sparse import bsr_matrix

indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6]).repeat(4).reshape(6, 2, 2)
bsr_matrix((data, indices, indptr), shape = (6, 6)).toarray()

array([[1, 1, 0, 0, 2, 2],
       [1, 1, 0, 0, 2, 2],
       [0, 0, 0, 0, 3, 3],
       [0, 0, 0, 0, 3, 3],
       [4, 4, 5, 5, 6, 6],
       [4, 4, 5, 5, 6, 6]])

## COO - Coordinate Sparse Matrix

**Advantages of the CSC format**

    * facilitates fast conversion among sparse formats
    * permits duplicate entries (see example)
    * very fast conversion to and from CSR/CSC formats

**Disadvantages of the CSC format**

    * does not directly support arithmetic operations and slicing
    
** Intended Usage**

    * COO is a fast format for constructing sparse matrices
    * Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector
    operations
    * By default when converting to CSR or CSC format, duplicate (i,j) entries will be summed together. 
    This facilitates efficient construction of finite element matrices and the like.


## DOK - Dictionary of Keys

Sparse matrices can be used in arithmetic operations: they support addition, subtraction, multiplication, division, and matrix power.

Allows for efficient O(1) access of individual elements. Duplicates are not allowed. Can be efficiently converted to a coo_matrix once constructed.

In [10]:
from scipy.sparse import dok_matrix
S = dok_matrix((5, 5), dtype=np.float32)
for i in range(5):
    for j in range(i, 5):
        S[i,j] = i+j
        
S.toarray()

array([[0., 1., 2., 3., 4.],
       [0., 2., 3., 4., 5.],
       [0., 0., 4., 5., 6.],
       [0., 0., 0., 6., 7.],
       [0., 0., 0., 0., 8.]], dtype=float32)

The ``scipy.sparse`` submodule also has a lot of functions for sparse matrices
including linear algebra, sparse solvers, graph algorithms, and much more.

# Exercises

<b>Exercise 1</b><br>
Create a big numpy **dense** matrix filled with random numbers in 
`[0, 1)`.
Generate a random number within this range and subsitute all the elements in the matrix **less than** this number with a zero.

Save resulting matrix as a `DOK` sparse matrix

In [15]:
d_matrix = np.random.randint(0, 1+1, size=(10, 10))

print(d_matrix)

[[0 1 0 0 1 1 1 0 1 0]
 [0 1 1 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0 1 1 1]
 [0 0 0 1 0 0 1 0 1 1]
 [1 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 1 1 1 1 0 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 0 1]]


In [16]:
s_matrix = dok_matrix(d_matrix, dtype=int)

print(s_matrix)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 8)	1
  (1, 1)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (2, 4)	1
  (2, 8)	1
  (3, 1)	1
  (3, 7)	1
  (3, 8)	1
  (3, 9)	1
  (4, 3)	1
  (4, 6)	1
  (4, 8)	1
  (4, 9)	1
  (5, 0)	1
  (5, 8)	1
  (6, 0)	1
  (6, 4)	1
  (6, 5)	1
  (6, 6)	1
  (6, 7)	1
  (6, 9)	1
  (7, 2)	1
  (7, 3)	1
  (7, 4)	1
  (7, 5)	1
  (7, 6)	1
  (7, 7)	1
  (7, 8)	1
  (7, 9)	1
  (8, 1)	1
  (8, 2)	1
  (8, 4)	1
  (8, 7)	1
  (9, 2)	1
  (9, 4)	1
  (9, 9)	1


In [24]:
d_matrix = np.random.random((1000, 1000))
print(d_matrix)

[[0.55547669 0.87281832 0.95857989 ... 0.83116461 0.8163194  0.64193517]
 [0.55812498 0.66478631 0.09978254 ... 0.39508448 0.53552437 0.88174867]
 [0.89355547 0.59564707 0.22490855 ... 0.2920441  0.1757782  0.71023137]
 ...
 [0.30173655 0.64222103 0.66182431 ... 0.72580734 0.05458402 0.38957938]
 [0.98217717 0.43915492 0.23936619 ... 0.9160611  0.01136096 0.88036203]
 [0.02849303 0.38233113 0.06061678 ... 0.01293167 0.85728161 0.538675  ]]


In [28]:
random_number = np.random.random()
random_number

0.5818051102694008

In [30]:
d_matrix[d_matrix < random_number] = 0
d_matrix

array([[0.        , 0.87281832, 0.95857989, ..., 0.83116461, 0.8163194 ,
        0.64193517],
       [0.        , 0.66478631, 0.        , ..., 0.        , 0.        ,
        0.88174867],
       [0.89355547, 0.59564707, 0.        , ..., 0.        , 0.        ,
        0.71023137],
       ...,
       [0.        , 0.64222103, 0.66182431, ..., 0.72580734, 0.        ,
        0.        ],
       [0.98217717, 0.        , 0.        , ..., 0.9160611 , 0.        ,
        0.88036203],
       [0.        , 0.        , 0.        , ..., 0.        , 0.85728161,
        0.        ]])

In [33]:
s_matrix = dok_matrix(d_matrix, dtype=float)
print(s_matrix)

  (0, 1)	0.8728183243775972
  (0, 2)	0.9585798908215738
  (0, 4)	0.7023167941145017
  (0, 5)	0.8524742319283757
  (0, 6)	0.9577710325536243
  (0, 11)	0.7678328175003856
  (0, 12)	0.9627914688248159
  (0, 13)	0.8140461695331237
  (0, 20)	0.5881274556775764
  (0, 21)	0.973705670703746
  (0, 24)	0.604395702009637
  (0, 25)	0.7370174602123152
  (0, 27)	0.9309763620260605
  (0, 29)	0.9769278822197447
  (0, 30)	0.9691784339640204
  (0, 31)	0.6125523027975711
  (0, 32)	0.7872907666310048
  (0, 33)	0.8551445501459298
  (0, 38)	0.6863060798034413
  (0, 46)	0.901673791049412
  (0, 47)	0.8405376221785964
  (0, 51)	0.9759999844001394
  (0, 52)	0.6209612354934748
  (0, 53)	0.9856844386289018
  (0, 55)	0.809910614543408
  :	:
  (999, 943)	0.9619965129577986
  (999, 944)	0.885034784052505
  (999, 946)	0.8886166077681078
  (999, 947)	0.9469983034896864
  (999, 950)	0.8109674062610072
  (999, 951)	0.6089504313402411
  (999, 952)	0.8487706038395261
  (999, 953)	0.6480701316272058
  (999, 954)	0.86443187

<b>Exercise 2</b><br>
Repeat the previous exercise, but this time use a `CSR` sparse matrix.

In [44]:
d_matrix = np.random.random((5, 5))
print(d_matrix)

[[0.45078513 0.49050126 0.46496364 0.66378595 0.28411874]
 [0.90364404 0.18479607 0.65506356 0.10182841 0.20726893]
 [0.15414533 0.53687954 0.41977928 0.66644541 0.85125072]
 [0.07073046 0.31067705 0.35445598 0.1799998  0.49397314]
 [0.9090427  0.99259163 0.50724062 0.53058238 0.53652927]]


In [40]:
random_number = np.random.random()
print(random_number)

0.4571099599505055


In [45]:
d_matrix[d_matrix < random_number] = 0
print(d_matrix)

[[0.         0.49050126 0.46496364 0.66378595 0.        ]
 [0.90364404 0.         0.65506356 0.         0.        ]
 [0.         0.53687954 0.         0.66644541 0.85125072]
 [0.         0.         0.         0.         0.49397314]
 [0.9090427  0.99259163 0.50724062 0.53058238 0.53652927]]


In [46]:
s_matrix = sparse.csr_matrix(d_matrix)
print(s_matrix)

  (0, 1)	0.49050125869197114
  (0, 2)	0.46496363900881077
  (0, 3)	0.6637859537172558
  (1, 0)	0.9036440428229898
  (1, 2)	0.655063555780313
  (2, 1)	0.5368795429283474
  (2, 3)	0.6664454134119628
  (2, 4)	0.8512507195147891
  (3, 4)	0.49397313961714506
  (4, 0)	0.9090426970435095
  (4, 1)	0.992591633696689
  (4, 2)	0.5072406204295365
  (4, 3)	0.5305823835866837
  (4, 4)	0.5365292685941252


<b>Exercise 3</b><br>
Transform the previously generated sparse matrix back to a full dense `numpy.array`.

In [48]:
to_d_matrix = s_matrix.toarray()
print(to_d_matrix)

[[0.         0.49050126 0.46496364 0.66378595 0.        ]
 [0.90364404 0.         0.65506356 0.         0.        ]
 [0.         0.53687954 0.         0.66644541 0.85125072]
 [0.         0.         0.         0.         0.49397314]
 [0.9090427  0.99259163 0.50724062 0.53058238 0.53652927]]


<b>Exercise 4</b><br>
Generate two sparse Matrix and sum them together, choosing the most appropriate internal representation (i.e. `LIL`, `CSR`, `DOK`...).

In [14]:
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix

In [7]:
s_mat1 = np.random.randint(0, 1+1, (4, 5))
print(s_mat1)

[[0 1 1 0 1]
 [1 0 1 0 1]
 [1 1 1 0 0]
 [0 1 0 1 0]]


In [8]:
s_mat2 = np.random.randint(0, 2+1, (4, 5))
print(s_mat2)

[[0 1 0 1 2]
 [0 2 0 1 2]
 [1 0 1 2 2]
 [2 2 0 1 2]]


In [21]:
csr_mat1 = csr_matrix(s_mat1)
csr_mat2 = csr_matrix(s_mat2)

In [22]:
csr_mat3 = csr_mat1 + csr_mat2
print(csr_mat3)

  (0, 1)	2
  (0, 2)	1
  (0, 3)	1
  (0, 4)	3
  (1, 0)	1
  (1, 1)	2
  (1, 2)	1
  (1, 3)	1
  (1, 4)	3
  (2, 0)	2
  (2, 1)	1
  (2, 2)	2
  (2, 3)	2
  (2, 4)	2
  (3, 0)	2
  (3, 1)	3
  (3, 3)	2
  (3, 4)	2


In [23]:
print(csr_mat3.toarray())

[[0 2 1 1 3]
 [1 2 1 1 3]
 [2 1 2 2 2]
 [2 3 0 2 2]]
